In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv("podatki.csv")
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
display(df)
xs = df.drop(["y"], axis=1)
ys = df["y"]

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x22,x23,x24,x25,x26,x27,x28,x29,x30,y
308,2.0,2.14,5.0,9.0,9.69,32.33,18.0,50.0,18.0,19.78,...,108.0,9589.04,541.27,534.61,2.20,4.0,4.0,4.0,7.0,teamEdward
545,2.0,2.50,2.0,3.0,2.00,19.43,6.0,7.0,5.0,4.00,...,13.0,71.74,36.87,5.87,3.00,2.0,2.0,2.0,3.0,teamEdward
902,2.0,2.23,7.0,13.0,20.52,19.69,22.0,46.0,20.0,35.08,...,118.0,19356.95,587.15,1077.28,2.20,10.0,4.0,10.0,3.0,teamEdward
52,2.0,3.00,2.0,2.0,2.00,8.20,7.0,3.0,3.0,4.50,...,8.0,40.77,17.51,4.15,3.00,2.0,2.0,2.0,3.0,teamEdward
186,2.0,2.29,2.0,4.0,2.00,25.76,17.0,19.0,18.0,21.00,...,94.0,8579.24,453.43,478.51,2.25,2.0,2.0,2.0,3.0,teamEdward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,4.0,2.10,4.0,7.0,14.90,33.25,16.0,40.0,13.0,18.00,...,106.0,8001.44,501.96,446.41,2.33,6.0,4.0,6.0,3.0,teamJacob
763,2.0,2.50,3.0,2.0,35.33,10.00,8.0,7.0,3.0,5.00,...,10.0,74.00,26.00,6.00,3.00,2.0,2.0,2.0,3.0,teamEdward
835,2.0,2.50,2.0,3.0,2.00,17.63,12.0,13.0,8.0,7.00,...,22.0,392.69,80.14,23.70,2.50,2.0,2.0,2.0,3.0,teamEdward
559,2.0,2.40,2.0,3.0,2.00,19.72,12.0,15.0,18.0,12.83,...,50.0,2082.00,194.00,117.56,2.25,2.0,2.0,2.0,5.0,teamEdward


In [6]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(xs, ys, train_size=0.8)

# 1.1 Ročno
We perform a train-test split and tune the parameters on the training dataset. We attempt using kNN method, neural networks, naive Bayes model, SVC and the random forest. We attempt to tune each manually to achieve the best cross validation score

In [7]:
from sklearn.model_selection import cross_validate

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=70)
cross_validate(knn, xtrain, ytrain, cv=5)["test_score"].mean()

0.8544689119170984

In [9]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(50, 50), solver="sgd", random_state=1)
cross_validate(nn, xtrain, ytrain, cv=5)["test_score"].mean()

0.8534272452504318

In [10]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

nb = GaussianNB()
mnb = MultinomialNB()
print(cross_validate(nb, xtrain, ytrain, cv=5)["test_score"].mean())
print(cross_validate(mnb, xtrain, ytrain, cv=5)["test_score"].mean())

0.8336949481865286
0.8045768566493955


In [11]:
from sklearn.svm import SVC
svc = SVC(C=5)
cross_validate(svc, xtrain, ytrain, cv=5)["test_score"].mean()

0.85551597582038

In [12]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=80,random_state=1)
cross_validate(rfc, xtrain, ytrain, cv=5)["test_score"].mean()

0.8856973229706391

All results are relatively close, with the correct choice of hyperparameters. We observe the best results with RandomForestClassifier, SVC and kNN, however, the results are often within margin of error.

# 1.2 Avtomatizirano

In [44]:
from hyperopt import hp, tpe, fmin, Trials, space_eval

gnezdeni_prostor = {
    "algo": hp.choice("algo", [
    {
        "ime": "rfc",
        "n_estimators": hp.choice("n_estimators", [16, 32, 64, 128, 256]),
        'max_depth': hp.choice('max_depth', [2, 4, 8, 16, 32]),
        "criterion": hp.choice("criterion", ["gini", "entropy", "log_loss"]),
        "min_samples_split": hp.qloguniform("min_samples_split", 1, 8, 1)
    },
    {
        "ime": "svc",
        "C": hp.lognormal("C", 0, 1),
        "shrinking": hp.choice("shrinking", [True, False]),
        "kernel": hp.choice("kernel", [
#            {
#                "type": "poly",
#                "gamma": hp.lognormal("gamma1", 1/30, 1),
#                "degree": hp.choice("degree", [2,3,4,5,6,8,12])
#            },
            {
                "type": "rbf",
                "gamma": hp.lognormal("gamma2", 1/30, 1)
            },
            {
                "type": "sigmoid",
                "gamma": hp.lognormal("gamma3", 1/30, 1)
            },
        ])
    },
    {
        "ime": "knn",
        "n_neighbors": hp.choice("n_neighbors", [2,4,8,16,32,64, 128, 196, 256]),
        "weights": hp.choice("weights", ["uniform", "distance"]),
        "p": hp.choice("p", [1, 2, 3, 4]),
    }])
}

In [45]:
def search_rfc(params):
    return RandomForestClassifier(
        n_estimators=int(params["n_estimators"]),
        criterion=params["criterion"],
        max_depth=params["max_depth"],
        min_samples_split=int(params["min_samples_split"]+1),
        random_state=1
    )

def search_svc(params):
    C = params["C"]
    shrinking = params["shrinking"]
    if params["kernel"]["type"] == "poly":
        return SVC(C=C, kernel="poly", gamma=params["kernel"]["gamma"],
                    degree=int(params["kernel"]["degree"]), shrinking=shrinking)
    if params["kernel"]["type"] == "rbf":
        return SVC(C=C, kernel="rbf", gamma=params["kernel"]["gamma"], shrinking=shrinking)
    return SVC(C=C, shrinking=shrinking, kernel="sigmoid", gamma=params["kernel"]["gamma"])

def search_knn(params):
    return KNeighborsClassifier(n_neighbors=int(params["n_neighbors"]), weights=params["weights"], p=params["p"])

def algorithms(params):
    x = params["algo"]
    name = x["ime"]
    model = None # Not needed due to python's scoping, polite.
    if name == "rfc":
        model = search_rfc(x)
    elif name == "knn":
        model = search_knn(x)
    else:
        model = search_svc(x)

    return 1-cross_validate(model, xtrain, ytrain, cv=5)["test_score"].mean()

In [46]:
trials = Trials()
best = fmin(fn=algorithms,
            space=gnezdeni_prostor,
            algo=tpe.suggest,
            max_evals=250,
            trials=trials)
best = space_eval(gnezdeni_prostor, best)
best_value = algorithms(best)
display(best, 1-best_value)

100%|██████████| 500/500 [02:09<00:00,  3.86trial/s, best loss: 0.10078799654576864]


{'algo': {'criterion': 'gini',
  'ime': 'rfc',
  'max_depth': 8,
  'min_samples_split': 12.0,
  'n_estimators': 32}}

0.8992120034542314

roc krivulja